In [101]:
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re

def cleaning(string):
    string = unidecode(string)
    return re.sub(r'[ ]+', ' ', string).strip()

In [43]:
r = requests.get('https://tatabahasabm.tripod.com/latih/latih.htm')
soup = BeautifulSoup(r.content, 'html.parser')
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if len(a_.get('href')) > 3 and a_.get('href').endswith('.htm')]
a = [a_ for a_ in a if not a_.startswith('http') and not a_.startswith('..')]
a = list(set(a))
len(a)

32

In [45]:
from tqdm import tqdm

nested = []
for a_ in tqdm(a):
    url = f'https://tatabahasabm.tripod.com/latih/{a_}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    frames = soup.find_all('frame')
    frames = [f.get('src') for f in frames]
    nested.extend(frames)
nested = list(set(nested))
len(nested)

100%|███████████████████████████████████████████| 32/32 [01:10<00:00,  2.20s/it]


60

In [48]:
for a_ in tqdm(nested):
    r = requests.get(f'https://tatabahasabm.tripod.com/latih/{a_}')
    with open(a_, 'w') as fopen:
        fopen.write(r.content.decode())

100%|███████████████████████████████████████████| 60/60 [01:38<00:00,  1.64s/it]


In [168]:
from glob import glob

files = glob('*.htm')
quizzes = []
for f in tqdm(files):
    with open(f) as fopen:
        content = fopen.read()
    soup = BeautifulSoup(content, 'html.parser')
    quiz = soup.find('form', {'name': 'QuizForm'})
    if quiz is None:
        continue
    splitted = str(quiz).split('<td colspan="4"><hr height="5" width="100%"/>')[1:]
    first = True
    instruction = None
    for s in splitted:
        try:
            s_ = s.split('<td></td>')
            q = BeautifulSoup(s_[0], 'html.parser').find_all('td')[-1].find('font').contents
            q = '\n'.join([str(q_) for q_ in q]).strip()
            if first:
                if '\n' in q:
                    instruction = q.split('\n')[0].strip()
                    q = q.split('\n')[1].strip()
                first = False
            q = cleaning(q.replace('\n', ''))
            choices = {}
            for s__ in s_[1:]:
                tds = BeautifulSoup(s__, 'html.parser').find_all('td')
                choice = tds[1].find('input').get('value').strip()
                answer = tds[1].find('input').get('onclick')[-2] == '1'
                text = cleaning(tds[2].text.strip())
                choices[choice] = {
                    'text': text,
                    'answer': answer
                }
            quizzes.append({
                'question': q,
                'instruction': instruction,
                'choices': choices,
                'website': f'https://tatabahasabm.tripod.com/latih/{f}'
            })
        except Exception as e:
            print(e)
            pass
    print(f, len(quizzes))

 28%|███████████▊                              | 26/92 [00:00<00:00, 256.18it/s]

list index out of range
kseruc.htm 5
list index out of range
soaljawabc.htm 25
list index out of range
lawanc.htm 45
list index out of range
ertic.htm 65


 77%|████████████████████████████████▍         | 71/92 [00:00<00:00, 149.30it/s]

list index out of range
hubungc.htm 111
list index out of range
betulc.htm 121
list index out of range
simpulc.htm 174
list index out of range
tandabacac.htm 180
list index out of range
majmukc.htm 190
list index out of range
gantidiric.htm 196
kmakna.htm 196


100%|██████████████████████████████████████████| 92/92 [00:00<00:00, 119.56it/s]

list index out of range
imbuhc.htm 300
list index out of range
peribahasac.htm 309
list index out of range
kgandac.htm 329
list index out of range
jodohc.htm 349
ainti.htm 349


In [169]:
quizzes

[{'question': '........, sudah dapat memandu kereta rupa-rupanya kamu !',
  'instruction': None,
  'choices': {'A': {'text': 'Oh', 'answer': False},
   'B': {'text': 'Eh', 'answer': True},
   'C': {'text': 'Hai', 'answer': False},
   'D': {'text': 'Ah', 'answer': False}},
  'website': 'https://tatabahasabm.tripod.com/latih/kseruc.htm'},
 {'question': '........... merahnya bibir budak perempuan itu !',
  'instruction': None,
  'choices': {'A': {'text': 'Aduhai', 'answer': False},
   'B': {'text': 'Ah', 'answer': False},
   'C': {'text': 'Amboi', 'answer': True},
   'D': {'text': 'Nah', 'answer': False}},
  'website': 'https://tatabahasabm.tripod.com/latih/kseruc.htm'},
 {'question': '............. , bisanya sengatan tebuan itu !',
  'instruction': None,
  'choices': {'A': {'text': 'Wah', 'answer': False},
   'B': {'text': 'Nah', 'answer': False},
   'C': {'text': 'Eh', 'answer': False},
   'D': {'text': 'Aduh', 'answer': True}},
  'website': 'https://tatabahasabm.tripod.com/latih/kseruc

In [171]:
import json

with open('quiz-tatabahasa.jsonl', 'w') as fopen:
    for q in quizzes:
        fopen.write(f'{json.dumps(q)}\n')